In [8]:
import os
import pandas as pd
import numpy as np
import ast
from PIL import Image

In [3]:
df = pd.read_csv("/raid/s2198939/MIMIC_Dataset/physionet.org/files/mimic-cxr-jpg/2.0.0/LLavA-Rad-Annotations/ANNOTATED_CSV_FILES/LLAVARAD_ANNOTATIONS_TRAIN.csv")
df.head()

,id,path,generate_method,reason,impression,indication,history,view,orientation,chexpert_labels,conversations,annotated_prompt,patientID,File_Exists,chexpert_labels_len
0,1000003250414267,files/p10/p10000032/s50414267/02aa804e-bde0afd...,gpt4,Eval for infection,No acute cardiopulmonary process.,NaN,NaN,PA,Erect,"{'Atelectasis': None, 'Cardiomegaly': None, 'C...","[{'from': 'human', 'value': '<image>\nDescribe...","There is no focal consolidation, pleural effus...",p10000032,1,14
1,1000003253189527,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,gpt4,___f with shortness of breath,No acute cardiopulmonary abnormality.,NaN,NaN,PA,Erect,"{'Atelectasis': None, 'Cardiomegaly': None, 'C...","[{'from': 'human', 'value': '<image>\nDescribe...","The cardiac, mediastinal and hilar contours ar...",p10000032,1,14
2,1000003253911762,files/p10/p10000032/s53911762/68b5c4b1-227d048...,gpt4,___f with cough // acute process?,No acute intrathoracic process.,NaN,NaN,AP,NaN,"{'Atelectasis': None, 'Cardiomegaly': None, 'C...","[{'from': 'human', 'value': '<image>\nDescribe...",Single frontal view of the chest provided. The...,p10000032,1,14
3,1000003253911762,files/p10/p10000032/s53911762/fffabebf-74fd3a1...,gpt4,___f with cough // acute process?,No acute intrathoracic process.,NaN,NaN,AP,Erect,"{'Atelectasis': None, 'Cardiomegaly': None, 'C...","[{'from': 'human', 'value': '<image>\nDescribe...",Single frontal view of the chest provided. The...,p10000032,1,14
4,1000003256699142,files/p10/p10000032/s56699142/ea030e7a-2e3b134...,gpt4,___ year old woman with cirrhosis.,No acute cardiopulmonary process.,NaN,NaN,AP,NaN,"{'Atelectasis': None, 'Cardiomegaly': None, 'C...","[{'from': 'human', 'value': '<image>\nDescribe...",Frontal chest radiographs were obtained with t...,p10000032,1,14


In [7]:
ROOT_PATH = "/raid/s2198939/MIMIC_Dataset/physionet.org/files/mimic-cxr-jpg/2.0.0"
df['path'] = df['path'].apply(lambda x: os.path.join(ROOT_PATH, x))
df['conversations'] = df['conversations'].apply(lambda x: ast.literal_eval(x))
df['annotated_prompt'] = df['conversations'].apply(lambda x: x[1]['value'])

In [9]:
df['conversations'][0]

[{'from': 'human',
  'value': '<image>\nDescribe the findings of the chest x-ray.\n'},
 {'from': 'gpt',
  'value': 'There is no focal consolidation, pleural effusion or pneumothorax. Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal. Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.'}]

In [12]:
"""
Create a yaml file of the following format dictionary

{
    "image_path": "path/to/your/image",
    "prompt": "a description of the image"
}

"""
def create_yaml_file(df, output_path):
    data = []
    for index, row in df.iterrows():
        image_path = row['path']
        prompt = row['annotated_prompt']
        
        # Check if the image exists
        if os.path.exists(image_path):
            data.append({
                "image_path": image_path,
                "prompt": prompt
            })
        else:
            print(f"Image not found: {image_path}")
    
    # Save to YAML file
    with open(output_path, 'w') as f:
        for item in data:
            f.write(f"- image_path: {item['image_path']}\n")
            f.write(f"  prompt: {item['prompt']}\n")

# Example usage
output_path = "/raid/s2198939/MIMIC_Dataset/physionet.org/files/mimic-cxr-jpg/2.0.0/LLavA-Rad-Annotations/lumina_data_train.yaml"
create_yaml_file(df, output_path)
# Load the YAML file to verify

